<a href="https://colab.research.google.com/github/cmj-dev/groomProject/blob/main/styleTokenMask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 97% 441M/457M [00:16<00:00, 19.8MB/s]
100% 457M/457M [00:17<00:00, 28.1MB/s]


In [ ]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  
  inflating: modelSource/class.json  
  inflating: modelSource/data/test   
  inflating: modelSource/data/test5  
  inflating: modelSource/data/train  
  inflating: modelSource/data/train5  
  inflating: modelSource/data/val    
  inflating: modelSource/data/val5   
  inflating: modelSource/emodel-class.log  
  inflating: modelSource/model/README.md  
  inflating: modelSource/model/config.json  
  inflating: modelSource/model/pytorch_model.bin  
  inflating: modelSource/model/tokenizer_config.json  
  inflating: modelSource/model/vocab.txt  
  inflating: modelSource/multi.json  
  inflating: modelSource/processor.py  
  inflating: modelSource/requir

In [ ]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

## readjson

In [ ]:
from pathlib import Path
import json
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    intensities = []
    for group in squad_dict:
        for passage in group['sentences']:
            text = passage['origin_text']
            intensity = passage['intensity']
            if intensity == 1:
                continue
            texts.append(text)
            if 
            intensities.append(intensity)

    return texts ,intensities

In [ ]:
import pandas as pd

file_path =  '/content/data/Training/talksets-train-1.json'

texts , intensities = read_aihub(file_path)
ai_df = pd.DataFrame(list(zip(texts , intensities)),
               columns =['texts','intensities'])

file_path = '/content/data/Validation/talksets-train-6.json'

texts , intensities = read_aihub(file_path)
val_df = pd.DataFrame(list(zip(texts , intensities)),
               columns =['texts','intensities'])

<_io.BufferedReader name='/content/data/Training/talksets-train-1.json'>
<_io.BufferedReader name='/content/data/Validation/talksets-train-6.json'>


In [ ]:
ai_df

,texts,intensities
0,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,2.0
1,맞아. 사람들이 진짜 개념이없다,2.0
2,저렇게는 살지 말아야지,2.0
3,그러게 게으르고 멍청한 사람들은 맞아야해,2.0
4,특히 벗방보는 애들은 진짜 거세 시켜야함,2.0
...,...,...
54172,앰생이 같은 게 같이 다녀줬더니 내가 니 친구로 보이냐?,1.8
54173,누가 누굴 데리고 다녀 나사 빠진 놈이,1.8
54174,너 오늘 나한테 처갈피는 줄 알아라,2.0
54175,엄친아들 다 없애버리고 싶다,1.6


In [ ]:
val_df = val_df.sample(frac=0.30,random_state=43)

In [ ]:
val_df

,texts,intensities
22120,닌 보는 영화보는 것 마다 그 소리하더라,0.0
14079,학과 조교 일하는데 너무 힘들다,0.0
27756,깡촌 아재들 여자 사들이는 중임?,1.4
31336,야 저 여자… 완전 쌩얼에서 풀메 완성하는데 15분 걸렸어.,0.0
26493,저는 그런 사람 만나봤는데 개념이 출타했나 생각들더라구요,1.6
...,...,...
20968,에휴 열심히 돈 벌어다 주면 뭐햐냐,0.0
14618,하지 마; 성괴 되고싶냐? 너 성형하면 성괴라고 내가 등에 써 붙일거임 ㅋ,1.6
550,싹 다 삼청교육대 끌고 가면 정신 차릴텐데 아쉽네,2.0
16458,너 보라고 한 애교 아니야.,0.0


In [ ]:
from datasets import Dataset

train_data = Dataset.from_pandas(ai_df)
dev_data = Dataset.from_pandas(val_df)
dev_data

Dataset({
    features: ['texts', 'intensities', '__index_level_0__'],
    num_rows: 11196
})

In [ ]:
train_data

Dataset({
    features: ['texts', 'intensities'],
    num_rows: 54177
})

## model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
model = AutoModelForSequenceClassification.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer(train_data[0]['texts'], return_tensors='pt')

{'input_ids': tensor([[ 9064,  7392,  7669,  7392, 12583,  9668, 25779,  9183,  8073, 25830,
         10135, 21968,  9136,  9126,  7098]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}